# Lab 5: Grouping, Aggregation, Pivot Tables, and DataFrame Manipulation in Pandas

This lab focuses on how to summarize, reshape, and manipulate tabular and time-series data using Pandas.

## Learning Objectives
- Understand and apply `groupby()` and `.agg()` for data summarization
- Use `.resample()` for time-based grouping
- Create and manipulate pivot tables
- Modify DataFrames by adding, transforming, and filtering columns
- Sort and rank data based on values
- Practice using these techniques in exercises

## 1. Create a Time Series Dataset
We begin by creating a small dataset simulating greenhouse sensor data, including temperature and CO2 levels over 12 days.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
date_rng = pd.date_range(start='2024-01-01', periods=12, freq='D')
data = {
    'Date': date_rng,
    'Greenhouse': ['A', 'B', 'A', 'C', 'A', 'B', 'C', 'A', 'C', 'B', 'A', 'C'],
    'Temperature': np.random.randint(20, 30, size=12),
    'CO2': np.random.randint(400, 450, size=12)
}
df = pd.DataFrame(data)
df.set_index('Date', inplace=True)
df

## 2. Grouping with `groupby()`
`groupby()` allows you to group data by categories (e.g., greenhouse ID) and compute summary statistics for each group.

In [ ]:
df.groupby('Greenhouse')[['Temperature', 'CO2']].mean()

## 3. Multiple Aggregations with `.agg()`
With `.agg()`, you can apply multiple aggregation functions (like mean, max, min, std) to each group.

In [ ]:
df.groupby('Greenhouse').agg({
    'Temperature': ['mean', 'max', 'min'],
    'CO2': ['mean', 'std']
})

## 4. Time-Based Grouping with `resample()`
If your index is a datetime, you can group by time periods. Here, we calculate the weekly average.

In [ ]:
df.resample('W').mean(numeric_only=True)

## 5. Pivot Table
`pivot_table()` reshapes data so that one variable becomes columns. Here, we display temperature by greenhouse and date.

In [ ]:
pivot = pd.pivot_table(df, values='Temperature', index='Date', columns='Greenhouse')
pivot

## 6. Handling Missing Values in Pivot Tables
Fill missing combinations with a default value (e.g., 0).

In [ ]:
pivot.fillna(0)

## 7. Visualize Grouped and Pivoted Data
Data visualization can help identify trends and compare values more clearly.

In [ ]:
# Bar chart
df.groupby('Greenhouse')['Temperature'].mean().plot(kind='bar', title='Average Temperature by Greenhouse')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()

In [ ]:
# Line plot from pivot
pivot.plot(title='Temperature by Greenhouse Over Time', figsize=(10, 6))
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()

## 8. Filtering with Conditions Before Grouping
You can filter rows before applying a groupby, for example selecting only rows with CO2 < 430.

In [ ]:
df[df['CO2'] < 430].groupby('Greenhouse')['Temperature'].mean()

## 9. Adding and Modifying Columns
You can create new columns by applying arithmetic operations to existing ones. Let's convert Celsius to Fahrenheit.

In [ ]:
df['Temp_F'] = df['Temperature'] * 9/5 + 32
df[['Temperature', 'Temp_F']]

## 10. Using `.apply()` for Categorization
Define a custom function and use `.apply()` to classify values into categories like 'Low', 'Medium', and 'High'.

In [ ]:
def classify_temp(temp):
    if temp < 23:
        return 'Low'
    elif temp <= 26:
        return 'Medium'
    else:
        return 'High'

df['Temp_Level'] = df['Temperature'].apply(classify_temp)
df[['Temperature', 'Temp_Level']]

## 11. Sorting and Ranking
You can sort rows based on column values and assign a ranking for comparisons.

In [ ]:
df_sorted = df.sort_values(by='Temperature', ascending=False)
df_sorted[['Greenhouse', 'Temperature']]

In [ ]:
df['Temp_Rank'] = df['Temperature'].rank(ascending=False)
df[['Temperature', 'Temp_Rank']]

## 12. Filtering with Multiple Conditions
You can use logical operators to filter data. Remember to wrap each condition in parentheses.

In [ ]:
df[(df['Temperature'] > 25) & (df['CO2'] < 430)][['Greenhouse', 'Temperature', 'CO2']]

## 13. 📝 Exercises
Try solving the following using the techniques you've learned:

1. Group by greenhouse and count the number of records.
2. Add a column for temperature deviation from the mean temperature.
3. Filter rows where `Temp_Level` is 'High' and display their CO2 values.
4. Create a new column showing CO2 as a percentage of the maximum CO2.
5. Create a pivot table showing average CO2 by `Temp_Level` and `Greenhouse`.
6. Plot the pivot table from (5) using a bar chart.